In [1]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import sys

sys.path.append(str(Path().cwd().parent.parent.resolve()))

# Uncomment to get more debugging printouts:
"""
import logging

root = logging.getLogger()
root.setLevel(logging.DEBUG)

handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
root.addHandler(handler)
"""

"\nimport logging\n\nroot = logging.getLogger()\nroot.setLevel(logging.DEBUG)\n\nhandler = logging.StreamHandler(sys.stdout)\nhandler.setLevel(logging.DEBUG)\nformatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')\nhandler.setFormatter(formatter)\nroot.addHandler(handler)\n"

In [2]:
from trulens_eval.keys import *

KEY SET: OPENAI_API_KEY
KEY SET: PINECONE_API_KEY
KEY SET: PINECONE_ENV
KEY SET: HUGGINGFACE_API_KEY
KEY SET: SLACK_TOKEN
KEY SET: SLACK_SIGNING_SECRET
KEY SET: COHERE_API_KEY


In [3]:
# LLama Index starter example from: https://gpt-index.readthedocs.io/en/latest/getting_started/starter_example.html

from llama_index import GPTVectorStoreIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader('llama_index/data').load_data()
index = GPTVectorStoreIndex.from_documents(documents)

query_engine = index.as_query_engine()
# response = query_engine.query("What did the author do growing up?")
# print(response)

In [4]:
from trulens_eval import TruLlama, Tru, Query, Feedback
from trulens_eval import tru_feedback

tru = Tru()
Tru().reset_database()

proc = Tru().start_dashboard(force=True, _dev=Path.cwd().parent.parent)
thread = Tru().start_evaluator(restart=True)

Force stopping dashboard ...
Starting dashboard ...
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Accordion(children=(HBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://192.168.1.60:8501 .
Looking for things to do. Stop me with `tru.stop_evaluator()`.

In [5]:
import numpy as np

# Construct feedback functions.

hugs = tru_feedback.Huggingface()
openai = tru_feedback.OpenAI()

# Language match between question/answer.
f_lang_match = Feedback(hugs.language_match).on(
    text1=Query.RecordInput, text2=Query.RecordOutput
)

# Question/answer relevance between overall question and answer.
f_qa_relevance = Feedback(openai.relevance).on(
    prompt=Query.RecordInput, response=Query.RecordOutput
)

# Question/statement relevance between question and each context chunk.
f_qs_relevance = tru_feedback.Feedback(openai.qs_relevance).on(
    question=Query.RecordInput,
    statement=Query.Record.model.retriever.retrieve.rets[:].node.text
).aggregate(np.min)

feedbacks = [f_qs_relevance]

l = tru.Llama(engine=query_engine, feedbacks=feedbacks)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface api: 0requests [00:00, ?requests/s]

openai api: 0requests [00:00, ?requests/s]

✅ chain chain_hash_cd3c32ff081e5cd453fda56f8f2f2f84 -> default.sqlite
✅ feedback def. feedback_definition_hash_3f3612d855dc16f65d110286de8bdf24 -> default.sqlite


In [6]:
list(l.instrumented())

[(JSONPath(), ['engine']),
 (JSONPath()._retriever, ['retriever']),
 (JSONPath().retriever, ['retriever'])]

In [7]:
res, record = l.query_with_record("Which art schools did the author apply to?")

Looking for things to do. Stop me with `tru.stop_evaluator()`.

Token indices sequence length is longer than the specified maximum sequence length for this model (1796 > 1024). Running this sequence through the model will result in indexing errors


✅ record record_hash_c67551b373774ecec33df9a3b9f28290 from chain_hash_cd3c32ff081e5cd453fda56f8f2f2f84 -> default.sqlite


matching 1_10, final parts=|8|
Model result: 0.8|
matching 1_10, final parts=|1|
Model result: 0.1|
✅ feedback feedback_result_hash_a47bd896e0c0aae1604fde349e9d8109 on record_hash_c67551b373774ecec33df9a3b9f28290 -> default.sqlite
Looking for things to do. Stop me with `tru.stop_evaluator()`.Looking for things to do. Stop me with `tru.stop_evaluator()`.

In [ ]:
# record.dict()

In [8]:
res, record = l.query_with_record("How rich was the author's boss?")

✅ record record_hash_fa413b1f9fd5eae7c5155808c5262adb from chain_hash_cd3c32ff081e5cd453fda56f8f2f2f84 -> default.sqlite
Looking for things to do. Stop me with `tru.stop_evaluator()`.matching 1_10, final parts=|1|
Model result: 0.1|
matching 1_10, final parts=|1|
Model result: 0.1|
✅ feedback feedback_result_hash_fbd7f3e06c82be6c25fb01528d679a8e on record_hash_fa413b1f9fd5eae7c5155808c5262adb -> default.sqlite


In [27]:
from pprint import PrettyPrinter

from trulens_eval.tru_feedback import OpenAI
from trulens_eval.util import TP
pp = PrettyPrinter()

openai = OpenAI()

class Dummy:
    def window_relevance(self, question, statement, window_size=1024):
        window_spacing = window_size // 2

        pieces = []

        for i in range(0, len(statement), window_spacing):
            window = statement[i:i+window_size]
            pieces.append((question, window, TP().promise(lambda w: openai.qs_relevance(question=question, statement=w), window)))

        return list(map(lambda t: (t[0], t[1], t[2].get()), pieces))

dummy = Dummy()

question = "How rich was the author's boss?"
statement = """
few months. Nor had I changed my grad student lifestyle significantly since we started. So when Yahoo bought us it felt like going from rags to riches. Since we were going to California, I bought a car, a yellow 1998 VW GTI. I remember thinking that its leather seats alone were by far the most luxurious thing I owned.

The next year, from the summer of 1998 to the summer of 1999, must have been the least productive of my life. I didn't realize it at the time, but I was worn out from the effort and stress of running Viaweb. For a while after I got to California I tried to continue my usual m.o. of programming till 3 in the morning, but fatigue combined with Yahoo's prematurely aged culture and grim cube farm in Santa Clara gradually dragged me down. After a few months it felt disconcertingly like working at Interleaf.

Yahoo had given us a lot of options when they bought us. At the time I thought Yahoo was so overvalued that they'd never be worth anything, but to my astonishment the stock went up 5x in the next year. I hung on till the first chunk of options vested, then in the summer of 1999 I left. It had been so long since I'd painted anything that I'd half forgotten why I was doing this. My brain had been entirely full of software and men's shirts for 4 years. But I had done this to get rich so I could paint, I reminded myself, and now I was rich, so I should go paint.

When I said I was leaving, my boss at Yahoo had a long conversation with me about my plans. I told him all about the kinds of pictures I wanted to paint. At the time I was touched that he took such an interest in me. Now I realize it was because he thought I was lying. My options at that point were worth about $2 million a month. If I was leaving that kind of money on the table, it could only be to go and start some new startup, and if I did, I might take people with me. This was the height of the Internet Bubble, and Yahoo was ground zero of it. My boss was at that moment a billionaire. Leaving then to start a new startup must have seemed to him an insanely, and yet also plausibly, ambitious plan.

But I really was quitting to paint, and I started immediately. There was no time to lose. I'd already burned 4 years getting rich. Now when I talk to founders who are leaving after selling their companies, my advice is always the same: take a vacation. That's what I should have done, just gone off somewhere and done nothing for a month or two, but the idea never occurred to me.

So I tried to paint, but I just didn't seem to have any energy or ambition. Part of the problem was that I didn't know many people in California. I'd compounded this problem by buying a house up in the Santa Cruz Mountains, with a beautiful view but miles from anywhere. I stuck it out for a few more months, then in desperation I went back to New York, where unless you understand about rent control you'll be surprised to hear I still had my apartment, sealed up like a tomb of my old life. Idelle was in New York at least, and there were other people trying to paint there, even though I didn't know any of them.

When I got back to New York I resumed my old life, except now I was rich. It was as weird as it sounds. I resumed all my old patterns, except now there were doors where there hadn't been. Now when I was tired of walking, all I had to do was raise my hand, and (unless it was raining) a taxi would stop to pick me up. Now when I walked past charming little restaurants I could go in and order lunch. It was exciting for a while. Painting started to go better. I experimented with a new kind of still life where I'd paint one painting in the old way, then photograph it and print it, blown up, on canvas, and then use that as the underpainting for a second still life, painted from the same objects (which hopefully hadn't rotted yet).

Meanwhile I looked for an apartment to buy. Now I could actually choose what
"""

In [29]:
pp.pprint(dummy.window_relevance(question, statement, window_size=4096))

matching 1_10, final parts=|1|
Model result: 0.1|
matching 1_10, final parts=|1|
Model result: 0.1|
[("How rich was the author's boss?",
  '\n'
  'few months. Nor had I changed my grad student lifestyle significantly since '
  'we started. So when Yahoo bought us it felt like going from rags to riches. '
  'Since we were going to California, I bought a car, a yellow 1998 VW GTI. I '
  'remember thinking that its leather seats alone were by far the most '
  'luxurious thing I owned.\n'
  '\n'
  'The next year, from the summer of 1998 to the summer of 1999, must have '
  "been the least productive of my life. I didn't realize it at the time, but "
  'I was worn out from the effort and stress of running Viaweb. For a while '
  'after I got to California I tried to continue my usual m.o. of programming '
  "till 3 in the morning, but fatigue combined with Yahoo's prematurely aged "
  'culture and grim cube farm in Santa Clara gradually dragged me down. After '
  'a few months it felt disco

In [30]:
pp.pprint(dummy.window_relevance(question, statement, window_size=2048))

matching 1_10, final parts=|1|
Model result: 0.1|
matching 1_10, final parts=|1|
Model result: 0.1|
matching 1_10, final parts=|1|
Model result: 0.1|
matching 1_10, final parts=|1|
Model result: 0.1|
[("How rich was the author's boss?",
  '\n'
  'few months. Nor had I changed my grad student lifestyle significantly since '
  'we started. So when Yahoo bought us it felt like going from rags to riches. '
  'Since we were going to California, I bought a car, a yellow 1998 VW GTI. I '
  'remember thinking that its leather seats alone were by far the most '
  'luxurious thing I owned.\n'
  '\n'
  'The next year, from the summer of 1998 to the summer of 1999, must have '
  "been the least productive of my life. I didn't realize it at the time, but "
  'I was worn out from the effort and stress of running Viaweb. For a while '
  'after I got to California I tried to continue my usual m.o. of programming '
  "till 3 in the morning, but fatigue combined with Yahoo's prematurely aged "
  'cultur

In [31]:
pp.pprint(dummy.window_relevance(question, statement, window_size=1024))

matching 1_10, final parts=|1|
Model result: 0.1|
matching 1_10, final parts=|8|
Model result: 0.8|
matching 1_10, final parts=|1|
Model result: 0.1|
matching 1_10, final parts=|1|
Model result: 0.1|
matching 1_10, final parts=|1|
Model result: 0.1|
matching 1_10, final parts=|1|
Model result: 0.1|
matching 1_10, final parts=|1|
Model result: 0.1|
matching 1_10, final parts=|1|
Model result: 0.1|
[("How rich was the author's boss?",
  '\n'
  'few months. Nor had I changed my grad student lifestyle significantly since '
  'we started. So when Yahoo bought us it felt like going from rags to riches. '
  'Since we were going to California, I bought a car, a yellow 1998 VW GTI. I '
  'remember thinking that its leather seats alone were by far the most '
  'luxurious thing I owned.\n'
  '\n'
  'The next year, from the summer of 1998 to the summer of 1999, must have '
  "been the least productive of my life. I didn't realize it at the time, but "
  'I was worn out from the effort and stress o

In [32]:
pp.pprint(dummy.window_relevance(question, statement, window_size=512))

matching 1_10, final parts=|8|
Model result: 0.8|
matching 1_10, final parts=|2|
Model result: 0.2|
matching 1_10, final parts=|1|
Model result: 0.1|
matching 1_10, final parts=|8|
Model result: 0.8|
matching 1_10, final parts=|1|
Model result: 0.1|
matching 1_10, final parts=|1|
Model result: 0.1|
matching 1_10, final parts=|1|
Model result: 0.1|
matching 1_10, final parts=|1|
Model result: 0.1|
matching 1_10, final parts=|1|
Model result: 0.1|
matching 1_10, final parts=|1|
Model result: 0.1|
matching 1_10, final parts=|2|
Model result: 0.2|
matching 1_10, final parts=|1|
Model result: 0.1|
matching 1_10, final parts=|1|
Model result: 0.1|
matching 1_10, final parts=|1|
Model result: 0.1|
matching 1_10, final parts=|1|
Model result: 0.1|
matching 1_10, final parts=|1|
Model result: 0.1|
[("How rich was the author's boss?",
  '\n'
  'few months. Nor had I changed my grad student lifestyle significantly since '
  'we started. So when Yahoo bought us it felt like going from rags to rich